In [ ]:
!git clone https://github.com/huggingface/transformers \
&& cd transformers \

Cloning into 'transformers'...
remote: Enumerating objects: 159080, done.
remote: Counting objects: 100% (1529/1529), done.
remote: Compressing objects: 100% (833/833), done.
remote: Total 159080 (delta 914), reused 1087 (delta 631), pack-reused 157551
Receiving objects: 100% (159080/159080), 160.15 MiB | 22.03 MiB/s, done.
Resolving deltas: 100% (119112/119112), done.
/bin/bash: line 1: cd: too many arguments


In [ ]:
!pip install -q ./transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.7 MB/s eta 0:00:00


In [ ]:
import torch
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [ ]:
torch_device = 'cpu'
import google.colab.output

def bart_summarize(text, num_beams, length_penalty, max_length, min_length, no_repeat_ngram_size):

  text = text.replace('\n','')
  text_input_ids = tokenizer.batch_encode_plus([text], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
  summary_ids = model.generate(text_input_ids, num_beams=int(num_beams), length_penalty=float(length_penalty), max_length=int(max_length), min_length=int(min_length), no_repeat_ngram_size=int(no_repeat_ngram_size))
  summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
  return summary_txt

#register callback for Javascript
google.colab.output.register_callback('bart_summarize', bart_summarize)

**Front-end** **NOT MY WORK**


In [ ]:
from IPython.display import HTML

#spinner from https://codepen.io/vovchisko/pen/vROoYQ
spinner_css = """
<style>
@keyframes c-inline-spinner-kf {
  0% {
    transform: rotate(0deg);
  }
  100% {
    transform: rotate(360deg);
  }
}

.c-inline-spinner,
.c-inline-spinner:before {
  display: inline-block;
  width: 11px;
  height: 11px;
  transform-origin: 50%;
  border: 2px solid transparent;
  border-color: #74a8d0 #74a8d0 transparent transparent;
  border-radius: 50%;
  content: "";
  animation: linear c-inline-spinner-kf 300ms infinite;
  position: relative;
  vertical-align: inherit;
  line-height: inherit;
}
.c-inline-spinner {
  top: 3px;
  margin: 0 3px;
}
.c-inline-spinner:before {
  border-color: #74a8d0 #74a8d0 transparent transparent;
  position: absolute;
  left: -2px;
  top: -2px;
  border-style: solid;
}
</style>
"""

input_form = """
<link rel="stylesheet" href="https://unpkg.com/purecss@1.0.1/build/pure-min.css" integrity="sha384-oAOxQR6DkCoMliIh8yFnu25d7Eq/PHS21PClpwjOTeU2jRSq11vu66rf90/cZr47" crossorigin="anonymous">

<div style="background-color:aqua; border:solid #ccc; width:800px; padding:20px; color: black;">
<p><strong>BART</strong> </p>
<textarea id="main_textarea" cols="75" rows="20" placeholder="Paste your text here..." style="font-family: 'Liberation Serif', 'DejaVu Serif', Georgia, 'Times New Roman', Times, serif; font-size: 13pt; padding:10px;"></textarea><br>
<div class="pure-form pure-form-aligned">

    <p><a target="_blank" href='https://pastebin.com/raw/BMPcUS6v'>Try to summarize this example article</a></p>
    <div style="width: 300px; display: block; margin-left: auto !important; margin-right: auto !important;">
        <p><button class="pure-button pure-button-primary" style="font-size: 125%%;" onclick="summarize()">Summarize</button>
        <span class="c-inline-spinner" style="visibility: hidden;" id="spinner"></span></p>
    </div>
</div>
</div>
"""

javascript = """
<script type="text/Javascript">


       function saveTextAsFile(textToWrite, fileNameToSaveAs)
    {
    	var textFileAsBlob = new Blob([textToWrite], {type:'text/plain'});
    	var downloadLink = document.createElement("a");
    	downloadLink.download = fileNameToSaveAs;
    	downloadLink.innerHTML = "Download File";
    	if (window.webkitURL != null)
    	{
    		// Chrome allows the link to be clicked
    		// without actually adding it to the DOM.
    		downloadLink.href = window.webkitURL.createObjectURL(textFileAsBlob);
    	}
    	else
    	{
    		// Firefox requires the link to be added to the DOM
    		// before it can be clicked.
    		downloadLink.href = window.URL.createObjectURL(textFileAsBlob);
    		downloadLink.onclick = destroyClickedElement;
    		downloadLink.style.display = "none";
    		document.body.appendChild(downloadLink);
    	}

    	downloadLink.click();
    }


    function summarize(){

        var text = document.getElementById('main_textarea').value;
        var no_repeat_ngram_size = document.getElementById('no_repeat_ngram_size').value;
        var num_beams = document.getElementById('num_beams').value;
        var length_penalty = document.getElementById('length_penalty').value;
        var max_length = document.getElementById('max_length').value;
        var min_length = document.getElementById('min_length').value;

        var kernel = google.colab.kernel;

        var resultPromise = kernel.invokeFunction("bart_summarize", [text,num_beams,length_penalty,max_length,min_length,no_repeat_ngram_size]); // developer, look here
        resultPromise.then(
            function(result) {
              document.getElementById('main_textarea').value = 'da resultado';
              document.getElementById('main_textarea').value = result.data["text/plain"];
              document.getElementById('spinner').style = "visibility: hidden;";
              saveTextAsFile(result.data["text/plain"], 'summary.txt')
        }).catch(function(error){document.getElementById('main_textarea').value = error;});
        document.getElementById('spinner').style = "visibility: visible;";
    };
</script>
"""


HTML(spinner_css + input_form + javascript)